In [1]:
# 表示用
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook

In [2]:
import numpy as np
import datetime
import os
import copy
# Generate WindowFrame Function
from mymodule import window
import joblib
import pandas as pd

from tqdm import tqdm_notebook

# Parametor about Sensor
SensorNum=1 # Sensor number
#SensorAxis='AccX' # Axis

# Define Data Name
DATE= str( datetime.date.today() )+'/'

# Parametor about " Data Path"
StorePath = "/media/takeyama/Transfer/02_ActivityResearch/"

ParticipantsNum="Participants07/"
Version = "01_First/"
Time = "01_First/"
CommonName = 'Participants07-First-01'
LabelName = CommonName+'.csv'

TITLE = ParticipantsNum+Version

DataPath = StorePath+ParticipantsNum+Version+"RawData/"+Time

if not os.path.exists(StorePath+TITLE+'LabelData/'+Time): 
    os.makedirs(StorePath+TITLE+'LabelData/'+Time)
LabelDataPath=StorePath+TITLE+'LabelData/'+Time

if not os.path.exists(StorePath+TITLE+'dictionary/'+Time): 
    os.makedirs(StorePath+TITLE+'dictionary/'+Time)
DictionaryDataPath=StorePath+TITLE+'dictionary/'+Time

if not os.path.exists(StorePath+TITLE+'feature/'+Time): 
    os.makedirs(StorePath+TITLE+'feature/'+Time)
FeaturePath=StorePath+TITLE+'feature/'+Time

if not os.path.exists(StorePath+TITLE+'window/'+Time): 
    os.makedirs(StorePath+TITLE+'window/'+Time)
WindowDataPath=StorePath+TITLE+'window/'+Time

if not os.path.exists(StorePath+TITLE+DATE+'studyOutput/'+Time): 
    os.makedirs(StorePath+TITLE+DATE+'studyOutput/'+Time)
StudyOutputPath=StorePath+TITLE+DATE+'studyOutput/'+Time

if not os.path.exists(StorePath+TITLE+DATE+'parametor/'+Time): 
    os.makedirs(StorePath+TITLE+DATE+'parametor/'+Time)
ParametorPath=StorePath+TITLE+DATE+'parametor/'+Time

if not os.path.exists(StorePath+TITLE+DATE+'result/'+Time): 
    os.makedirs(StorePath+TITLE+DATE+'result/'+Time)
ResultPath=StorePath+TITLE+DATE+'result/'+Time

if not os.path.exists(StorePath+TITLE+DATE+'graph/'+Time): 
    os.makedirs(StorePath+TITLE+DATE+'graph/'+Time)
GraphPath=StorePath+TITLE+DATE+'graph/'+Time

# 1 making Sensor_Data_Array.npz

In [3]:
# センサデータの読み込み
# センサデータのcsvファイル名をリストに集約する関数
def SensorDataFileNameList(path):
    l = os.listdir(DataPath)
    FilePath =[]
    SensorName = []
    
    for n in range(len(l)):
        if (l[n][:4].find('mem-') != -1):
            FilePath.append(l[n])
            SensorName.append(l[n][4:15])
    
    return FilePath,SensorName

In [4]:
# CSVを読み込んで，CSVの中身を辞書型で表現する．
def ImportCSV(csv_file,SensorName,mode='Round'):
    import pandas as pd
    '''
    ~Argument~
    csv_file -> ファイル名 
    mode Round -> 四捨五入
         Roundup -> 切り上げ
         Rounddown -> 切り捨て
    
    ~Conversion~
    Acc Data  [0.1mG]=>[G]
    Gyr Data  [0.01dps]=>[dps]   ...dps=degree per second
    '''
    # data dictionary 
    RawData={}   
    AccConversion = 0.1 * 0.001
    GyrConversion = 0.01
    
    # design dataframe and import csv
    data = pd.read_csv(csv_file)
    data.columns=[u'Type',u'Time',u'AccX',u'AccY',u'AccZ',u'GyrX',u'GyrY',u'GyrZ']
    data = data[ data['Type']=='ags']

    # convert numpy.darray 

    AccX=data.AccX.values*AccConversion
    AccY=data.AccY.values*AccConversion
    AccZ=data.AccZ.values*AccConversion
    
    GyrX=data.GyrX.values*GyrConversion
    GyrY=data.GyrY.values*GyrConversion
    GyrZ=data.GyrZ.values*GyrConversion

    # regist each raw data 
    RawData['AccX'] = AccX
    RawData['AccY'] = AccY
    RawData['AccZ'] = AccZ
    RawData['GyrX'] = GyrX
    RawData['GyrY'] = GyrY
    RawData['GyrZ'] = GyrZ
    RawData['Name'] = SensorName

    RawData['Time'] = data.Time.values
    return RawData

Making sensor data and store the data

In [5]:
Label = ['01_LeftHand','02_RightHand','03_LeftLeg','04_Rightleg','05_West','06_Chest']

DataFileNameList,SensorName=SensorDataFileNameList(DataPath)
l=[]
for i in range( len(DataFileNameList) ):
    l.append( ImportCSV(DataPath+DataFileNameList[i],Label[i]) )

Sync the data's start and goal time to each other sensor data

In [6]:
def MakeCommonSection(inputDataArray):
    # 共通区間のスタート時間、ゴール時間を求める
    # スタート時間を求める
    startTime = min(inputDataArray[0]['Time'])
    
    for i in range(len(inputDataArray)):
        if startTime < min(inputDataArray[i]['Time']):
            startTime = min(inputDataArray[i]['Time'])
    
    # ゴール時間を求める
    goalTime = max(inputDataArray[0]['Time'])
    
    for i in range(len(inputDataArray)):
        if goalTime > max(inputDataArray[i]['Time']):
            goalTime = max(inputDataArray[i]['Time'])

    # 共通区間のスタート時間のインデックス、ゴール時間のインデックスを探索する
    def CalcSearchIndexFromTime(data, keyTime):
        """
        data　辞書型
        keyTime data['Time']の中の探す値
        """
        count = 0
        for i in range(0, len(data['Time'])):
            if keyTime == data['Time'][i]:
                print str(keyTime)+' is much in the index  whose number is '+str(i)
                return i
            
    startIndex = np.array([])
    goalIndex = np.array([])
    for obj in inputDataArray:
        print 'start'
        startIndex = np.append(startIndex, CalcSearchIndexFromTime(obj, startTime) ).astype(int)
        print 'goal'
        goalIndex = np.append(goalIndex, CalcSearchIndexFromTime(obj, goalTime) ).astype(int)

    tmp={}
    comDataArray =[]
    key={}

    # センサデータすべて（時刻、加速度、角速度）に対して共通区間のみのデータを抽出
    for number,iDA in enumerate( inputDataArray ):

        tmp['AccX'] = copy.deepcopy( iDA['AccX'][startIndex[number]:goalIndex[number]] )
        tmp['AccY'] = copy.deepcopy( iDA['AccY'][startIndex[number]:goalIndex[number]] )
        tmp['AccZ'] = copy.deepcopy( iDA['AccZ'][startIndex[number]:goalIndex[number]] )
        tmp['GyrX'] = copy.deepcopy( iDA['GyrX'][startIndex[number]:goalIndex[number]] )
        tmp['GyrY'] = copy.deepcopy( iDA['GyrY'][startIndex[number]:goalIndex[number]] )
        tmp['GyrZ'] = copy.deepcopy( iDA['GyrZ'][startIndex[number]:goalIndex[number]] )
        tmp['Time'] = copy.deepcopy( iDA['Time'][startIndex[number]:goalIndex[number]] )
        tmp['Name'] = copy.deepcopy( iDA['Name'] )
        comDataArray.append(copy.deepcopy(tmp) )    
        key[ tmp['Name'] ] = number
       
    return key,comDataArray

In [7]:
TrimKey,TrimData = MakeCommonSection(l)

start
47101062 is much in the index  whose number is 77
goal
47927899 is much in the index  whose number is 826914
start
47101062 is much in the index  whose number is 39
goal
47927899 is much in the index  whose number is 826876
start
47101062 is much in the index  whose number is 0
goal
47927899 is much in the index  whose number is 826837
start
47101062 is much in the index  whose number is 954
goal
47927899 is much in the index  whose number is 827791
start
47101062 is much in the index  whose number is 899
goal
47927899 is much in the index  whose number is 827736
start
47101062 is much in the index  whose number is 878
goal
47927899 is much in the index  whose number is 827715


store the data by using np.savez and change sensordata.csv to sensordata.npz

In [8]:
for i in range(6):
    np.savez(DictionaryDataPath+TrimData[i]['Name'],data=TrimData[i])

# 2 Making WindowFrame by using often used parameter

In [9]:
def GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH):
    """
    data　ウィンドウフレームに変換するデータ
    registName　ウィンドウフレームに登録するデータの名前
    windowWidth　ウィンドウ幅
    slidingWidth　スライド幅
    PATH ウィドウフレームを保存するディレクトリ
    
    
    About Function:
        与えられたdataからウィンドウ幅windowWidth,スライド幅slidingWidthにしたがって
        registNameのウィドウフレームを返す．
        また，与えれたPATH内に同様なパラメータ( WindowWidth, slidingWidth)かつ同様な
        windowNameのものがある場合，そのデータを返す．
        この関数が登録，ウィンドウフレームに変換できるデータは１つとする
    """
    storedName = windowName+'_Win='+str(wWidth).zfill(4)+'_Sld='+str(sWidth).zfill(4)+'.npz'
    l = os.listdir(PATH)
    
    if storedName in l:
        print "this data had finished making"
        return np.load(PATH+storedName)['data'][()]
    
    w=window()
    w.SetData(windowName,dataRaw)    
    wind=w.Compile(wWidth,sWidth)
    windoW=wind.reshape((len(wind),np.prod(wind.shape[1:])))
    
    np.savez(PATH+storedName+'.npz',data=wind)

    return wind

In [10]:
DictName=os.listdir(DictionaryDataPath)
print DictName
Sensor5 = np.load(DictionaryDataPath+DictName[0])['data'][()]
Sensor1 = np.load(DictionaryDataPath+DictName[1])['data'][()]
Sensor3 = np.load(DictionaryDataPath+DictName[2])['data'][()]
Sensor2 = np.load(DictionaryDataPath+DictName[3])['data'][()]
Sensor4 = np.load(DictionaryDataPath+DictName[4])['data'][()]
Sensor6 = np.load(DictionaryDataPath+DictName[5])['data'][()]

['01_LeftHand.npz', '02_RightHand.npz', '03_LeftLeg.npz', '04_Rightleg.npz', '05_West.npz', '06_Chest.npz', '07_LabelData.npz']


In [11]:
arrayData = [Sensor1,Sensor2,Sensor3,Sensor4,Sensor5,Sensor6]
arrayDataName =['Sensor1','Sensor2','Sensor3','Sensor4','Sensor5','Sensor6']

for ad,adn in zip(arrayData,arrayDataName):
    for axis in ['AccX','AccY','AccZ']:
        FileName = 'Participants07-First-01-'+adn+'-'+axis
        GetWindowFrame(FileName,ad[axis],256,32,WindowDataPath)

Participants07-First-01-Sensor1-AccX is registed now
Build Complete
Source shape is(826837, 1)
window frames num =25831
offset=21
(25831, 256, 1)
Participants07-First-01-Sensor1-AccY is registed now
Build Complete
Source shape is(826837, 1)
window frames num =25831
offset=21
(25831, 256, 1)
Participants07-First-01-Sensor1-AccZ is registed now
Build Complete
Source shape is(826837, 1)
window frames num =25831
offset=21
(25831, 256, 1)
Participants07-First-01-Sensor2-AccX is registed now
Build Complete
Source shape is(826837, 1)
window frames num =25831
offset=21
(25831, 256, 1)
Participants07-First-01-Sensor2-AccY is registed now
Build Complete
Source shape is(826837, 1)
window frames num =25831
offset=21
(25831, 256, 1)
Participants07-First-01-Sensor2-AccZ is registed now
Build Complete
Source shape is(826837, 1)
window frames num =25831
offset=21
(25831, 256, 1)
Participants07-First-01-Sensor3-AccX is registed now
Build Complete
Source shape is(826837, 1)
window frames num =25831
offs

# Making WindowFrame from Label Data

In [12]:
def MakeWindowFrameLabel(LabelData1d, wWidth, sWidth):
    w=window()
    w.SetData('DataAddingLabel', LabelData1d)    
    wind=w.Compile(wWidth, sWidth)
    windoW=wind.reshape((len(wind),np.prod(wind.shape[1:])))

    LabelArray=np.array([np.unique(windoW[l,:])[
            np.argmax(np.array([len( np.where(windoW[l]==np.unique(windoW[l])[i])[0])for i in range(np.unique(windoW[l]).size)]))]
             for l in range(windoW.shape[0])
    ])
    return LabelArray

In [13]:
def MakeTimeDomainLabelData(DataName,StartTime,GoalTime):
    # import LabelData -> pandas
    LabelDf = pd.read_csv(DataName,encoding='Shift-JIS')
    LabelData = LabelDf.as_matrix() #pandas -> ndarray
    
    # timedata hh.mm.ss -> ms
    TimeArray = pd.to_datetime( LabelDf['#Time'].values ) 
    t = np.array([ TimeArray[i].hour*3600*1000+TimeArray[i].minute*60*1000+TimeArray[i].second*1000+TimeArray[i].microsecond*0.001
                 for i in range( len(TimeArray) )]).astype(int)
    
    LabelData[:,0] = t.astype(int) #time data: unicode -> int
    # Time Series
    LabelArray = np.array([
        np.append( np.array(LabelData[i,0]+l) , LabelData[i,1:])
        for i in range(LabelData.shape[0]-1) 
        for l in range(LabelData[i+1,0]-LabelData[i,0])
    ])
    
    StartLabelArray = np.array([
        np.append( StartTime+i, LabelArray[0,1:])
        for i in range(LabelArray[0,0]-StartTime)
    ])
    
    GoalLabelArray = np.array([
        np.append( GoalTime+i, LabelArray[-1,1:])
        for i in range(1,1+GoalTime-LabelArray[-1,0])
    ])
    
    LabelArrayComp = np.vstack((StartLabelArray, LabelArray))
    LabelArrayComp = np.vstack((LabelArrayComp, GoalLabelArray))
    df = pd.DataFrame(LabelArrayComp)
    df.columns = LabelDf.columns
    
    return df.to_dict()

In [14]:
DictName = os.listdir(DictionaryDataPath)
print DictName
Sensor5 = np.load(DictionaryDataPath+DictName[0])['data'][()]
Sensor1 = np.load(DictionaryDataPath+DictName[1])['data'][()]
Sensor3 = np.load(DictionaryDataPath+DictName[2])['data'][()]
Sensor2 = np.load(DictionaryDataPath+DictName[3])['data'][()]
Sensor4 = np.load(DictionaryDataPath+DictName[4])['data'][()]
Sensor6 = np.load(DictionaryDataPath+DictName[5])['data'][()]

StartTime = Sensor1['Time'][0]
GoalTime = Sensor1['Time'][-1]

['01_LeftHand.npz', '02_RightHand.npz', '03_LeftLeg.npz', '04_Rightleg.npz', '05_West.npz', '06_Chest.npz', '07_LabelData.npz']


In [17]:
dictLabel=MakeTimeDomainLabelData(LabelDataPath+LabelName,StartTime,GoalTime)
np.savez(DictionaryDataPath+'07_LabelData',data=dictLabel)